## Install IRkernel

1. Install IRkernel in R
```R
install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'))
devtools::install_github('IRkernel/IRkernel')
```

2. Making the kerbel available for jupyter

```R
IRkernel::installspec()
```

## Valeurs retournées par la fonction `predict.cartgv` :

La fonction renvoie une matrice donnant pour chaque 
individu de new (dans l'ordre) :
  - `hat.Y` :la classe prédite
  - `Y` : la classe d'appartenance
  - `noeuds` : le numéro du noeud contenant l'observation
  - `score` : le score le l'observation


## Prediction de la feuille

 Fonction permettant de donner le nom cart et l'indice du noeud (sortie de la 
fonction `carts$where`) qui contient une nouvelle observation

*Paramètres d'entrée*:
- `new_obs` : une nouvelle observation (avec les mêmes variables que celle de 
           l'echantillon d'apprentissage)
- `tableau` : sortie de la fonction `calcul_cart()`

In [2]:
predict.cartgv <- function(new, tree, carts, coups) {
  P <- dim(new)[1]
  
  predgv <- function(ind) {
    i <- 1
    while (tree$action[which(tree$node == i)] >= 1) {
      tab <- coups[[i]]
      if (is.data.frame(tab)) {
        i_node <- 1
        while (tab$var[i_node] != "<leaf>") {
          i_sons <- which(tab$parent == tab$node_name_cart[i_node])
          if (ind[tab$var[i_node]] >= tab$threshold[i_node]) {
            if (tab$sens[i_node] == "sup")
              k <- 1
            else
              k <- 2
          } else{
            if (tab$sens[i_node] == "sup")
              k <- 2
            else
              k <- 1
          }
          i_node <- i_sons[k]
        }
        
        i <-  tree$node[which(
          tree$parent == i
          & tree$i_node_coupure == i_node
          & tree$yval == tab$pred[i_node]
          & tree$n_noncase == tab$n_noncase[i_node]
          & tree$n_case == tab$n_case[i_node]
        )]
      }
    }
    i.eq.tree.node <- which(tree$node == i)
    if (tree$action[i.eq.tree.node] < 1) {
      # on teste si le noeuds dans lequel tombre l'obs est une feuille
      return(c(tree$yval[i.eq.tree.node], i, tree$prob[i.eq.tree.node]))
    } else {
      return(c(NA, NA, NA))
    }
    
  }
  
  res <- matrix()
  length(res) <- 4 * P
  dim(res) <- c(P, 4)
  
  res[,1] <- new[,1]
  res[,2:4] <- t(apply(new,1,predgv))
  
  colnames(res) <- c("Y", "hat.Y", "noeuds", "score")
  
  return(as.data.frame(res))
}


## ETAPE 1
- charchement du fichier DataIris_test.RData
- chargement du fichier code.R

In [3]:
load(file = "../data/DataPhoneme_test.RData")

In [4]:
indx <- colnames(tree)
indx <- indx[which(indx != 'leave')]
tree[indx] <-
  lapply(tree[indx], function(x)
    as.numeric(as.character(x)))

coups <- lapply(coups, function(df) {
  indx <-
    c('pred',
      'parent',
      'node_name_cart',
      'threshold',
      'n_noncase',
      'n_case',
      'prob',
      'depth')
  df[indx] <-
    lapply(df[indx], function(x)
      as.numeric(as.character(x)))
  df['var'] <- lapply(df['var'], function(x)
    as.character(x))
  df
})

indx <- colnames(new)
new[indx]  <- lapply(new[indx], function(x) as.numeric(as.character(x)))

indx <- colnames(test)
test[indx] <- lapply(test[indx], function(x) as.numeric(as.character(x)))

In [5]:
predict.cartgv(new, tree, carts, coups)$hat.Y


[1] 1 1 1 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 [38] 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1
 [75] 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 0 0
[112] 0 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0
[149] 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1
[186] 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0
[223] 0 1 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
[260] 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1
[297] 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1

In [6]:
predict.cartgv(test, tree, carts, coups)$hat.Y

[1] 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 [38] 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0
 [75] 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0
[112] 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0
[149] 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0
[186] 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0
[223] 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0
[260] 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0
[297] 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0
[334] 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0
[371] 1 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1
[408] 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
[445] 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1
[482] 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
[519] 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1
[556] 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0
[593] 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0
[630] 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1
[667] 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1
[704] 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1
[741] 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1
[778] 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0
[815] 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0
[852] 1 0 1 1 0 1 0

In [7]:
indx <- colnames(tree)
indx <- indx[which(indx != 'leave')]
tree[indx] <-
  lapply(tree[indx], function(x)
    as.numeric(as.character(x)))
tree

action,var,depth,parent,n,n_case,n_noncase,yval,prob,leave,node,improvment,i_node_coupure
1,4,0,NA,835,438,397,1,0.5246,,1,56.7261,NA
1,3,1,1,253,131,122,1,0.5178,,2,63.5809,4
-1,NA,1,1,72,66,6,1,0.9167,*,3,NA,7
1,10,1,1,254,75,179,0,0.2953,,4,39.2332,3
1,2,1,1,256,166,90,1,0.6484,,5,72.7718,6
-1,NA,2,2,144,117,27,1,0.8125,*,6,NA,7
-1,NA,2,2,20,9,11,0,0.4500,*,7,NA,4
-1,NA,2,2,69,2,67,0,0.0290,*,8,NA,3
-1,NA,2,2,20,3,17,0,0.1500,*,9,NA,6
-1,NA,2,4,153,15,138,0,0.0980,*,10,NA,3
